# The Agentic Loop with Temporal - Dynamic Activities

In this notebook, we'll learn how to build an AI agent that can:
- Analyze a high-level goal
- Choose which tools to use with parameters
- Execute tools **dynamically at runtime** using Temporal's Dynamic Activities
- Complete complex tasks autonomously

## Key Innovation: Dynamic Activities
We use Temporal's **Dynamic Activities** feature to execute tools without pre-registering each one individually!

## What is an Agentic Loop?

An agentic loop is a pattern where an AI agent autonomously:
1. **Observes** - Analyzes the current state and goal
2. **Decides** - Chooses what action to take next
3. **Acts** - Executes the chosen action
4. **Reflects** - Evaluates progress and decides next steps

### Why Temporal for AI Agents?

**Durability**: Your AI agent continues working even if:
- The process crashes
- API calls fail
- The server restarts

**Observability**: Full visibility into:
- Every decision the AI makes
- Each tool execution
- The complete execution history

**Reliability**: Built-in features like:
- Automatic retries with exponential backoff
- Timeouts for long-running operations
- State persistence across failures

## Architecture Overview

Our system has a clean separation of concerns:

1. **`tools.py`** - Tool implementations and definitions
   - Actual tool functions (search_flights, book_flight, etc.)
   - Tool metadata for AI understanding
   - Tool registry for dynamic execution

2. **`activities.py`** - AI agent helpers
   - `AgentActivities.ai_select_tool_with_params` - AI decision making
   - `dynamic_tool_activity` - Dynamic handler for runtime tool execution

3. **`workflow.py`** - Orchestration logic
   - Agentic loop implementation
   - Dynamic activity invocation

4. **`models.py`** - Data structures
   - Tool definitions, arguments, workflow inputs/outputs

In [ ]:
# Set up environment
import asyncio
import inspect
import json
import os
import sys
from collections.abc import Callable
from datetime import datetime, timedelta
from typing import Awaitable, Sequence

# Add parent directories to path
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

# For running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

print("Setup complete!")

## Step 1: Define Data Models

We'll create structured models for our tools and workflow.

In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Union

@dataclass
class BookingRequest:
    """Input for our agent - just a high-level goal!"""
    goal: str  # e.g., "Book a flight from NYC to London on March 15"

@dataclass
class BookingResult:
    """Output from our agent - what it accomplished"""
    message: str
    steps_taken: List[str]

@dataclass
class ToolArgument:
    """Defines an argument for a tool."""
    name: str
    type: str  # "string", "integer", etc.
    description: str

@dataclass  
class ToolDefinition:
    """Structured definition of a tool for the AI agent."""
    name: str
    description: str
    arguments: List[ToolArgument]

print("Models defined!")

## Step 2: The AI Decision Activity

This is the "brain" of our agent - it decides which tool to use next AND extracts parameters!

In [ ]:
from temporalio import activity, workflow
from activities import AgentActivities

class AgentActivities:
    @activity.defn
    async def ai_select_tool_with_params(
        self, goal: str, available_tools: dict, context: str
    ) -> dict:
        """AI decides which tool to use AND extracts parameters."""
        
        # Import inside activity for Temporal safety
        with workflow.unsafe.imports_passed_through():
            from litellm import completion
            from config import OPENAI_API_KEY
        
        # Build tool descriptions
        tools_description = []
        for name, tool_def in available_tools.items():
            tool_str = f"Tool: {name}\n"
            tool_str += f"Description: {tool_def.description}\n"
            tool_str += "Arguments: " + ", ".join(
                [f"{arg.name} ({arg.type}): {arg.description}" 
                 for arg in tool_def.arguments]
            )
            tools_description.append(tool_str)
        
        tools_text = "\n\n".join(tools_description)
        
        prompt = f"""You are an AI agent working to achieve this goal: {goal}

Available tools:
{tools_text}

Current context:
{context if context else "Just starting - no actions taken yet"}

Return a JSON object with:
- "tool": the tool name to use (or "DONE" if complete)
- "parameters": an object with the required parameters

Examples:
{{"tool": "search_flights", "parameters": {{"origin": "NYC", "destination": "London", "date": "March 15"}}}}
{{"tool": "book_flight", "parameters": {{"flight_id": "AA123", "seat_class": "economy"}}}}
{{"tool": "DONE", "parameters": {{}}}}

Return ONLY the JSON object."""

        response = completion(
            model="openai/gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            api_key=OPENAI_API_KEY
        )
        
        decision_text = response.choices[0].message.content.strip()
        print(f"AI selected: {decision_text}")
        
        import json
        try:
            return json.loads(decision_text)
        except json.JSONDecodeError:
            if "DONE" in decision_text.upper():
                return {"tool": "DONE", "parameters": {}}
            return {"tool": decision_text, "parameters": {}}

print("AI decision activity defined!")

## Step 3: Define the Tools in tools.py

Tools are implemented in `tools.py` with both the implementation AND metadata.

# Example of tools.py structure
from tools import AVAILABLE_TOOLS, TOOL_HANDLERS

# Tool implementations
async def search_flights(params: dict) -> str:
    """Search for available flights."""
    origin = params.get("origin", "Unknown")
    destination = params.get("destination", "Unknown")
    date = params.get("date", "Unknown")
    
    return f"""Found 3 flights from {origin} to {destination} on {date}:
- Flight AA123: Departs 8:00 AM, arrives 8:00 PM, $450
- Flight UA456: Departs 2:15 PM, arrives 2:15 AM (+1 day), $380
- Flight DL789: Departs 6:30 PM, arrives 6:30 AM (+1 day), $420"""

async def book_flight(params: dict) -> str:
    """Book a specific flight."""
    flight_id = params.get("flight_id", "Unknown")
    seat_class = params.get("seat_class", "economy")
    
    confirmation = f"CONF-{flight_id}-{datetime.now().strftime('%Y%m%d%H%M%S')}"
    result = f"""Flight booked successfully!
- Flight: {flight_id}
- Class: {seat_class}  
- Confirmation: {confirmation}"""
    
    # Only add passenger if provided (no hardcoded names!)
    if passenger_name := params.get("passenger_name"):
        result += f"\n- Passenger: {passenger_name}"
    
    return result

# Tool registry maps names to implementations
TOOL_HANDLERS = {
    "search_flights": search_flights,
    "book_flight": book_flight,
    # ... other tools
}

print("Tools defined with clean separation!")

In [ ]:
from temporalio import activity, workflow

@activity.defn
async def ai_select_tool_with_params(
    goal: str, 
    available_tools: Dict[str, ToolDefinition], 
    context: str
) -> Dict[str, Union[str, Dict]]:
    """AI selects tool AND extracts parameters in one call."""
    
    # Import inside activity for Temporal safety
    with workflow.unsafe.imports_passed_through():
        from litellm import completion
        from config import OPENAI_API_KEY
    
    # Build tool descriptions
    tools_description = []
    for name, tool_def in available_tools.items():
        tool_str = f"Tool: {name}\n"
        tool_str += f"Description: {tool_def.description}\n"
        tool_str += "Arguments: " + ", ".join(
            [f"{arg.name} ({arg.type}): {arg.description}" for arg in tool_def.arguments]
        )
        tools_description.append(tool_str)
    
    tools_text = "\n\n".join(tools_description)
    
    prompt = f"""You are an AI agent working to achieve this goal: {goal}

Available tools:
{tools_text}

Current context:
{context if context else "Just starting - no actions taken yet"}

Based on the goal and context, decide the next action.
Extract any parameters from the goal or context.

Return a JSON object with:
- "tool": the tool name to use (or "DONE" if complete)
- "parameters": an object with the required parameters

Example responses:
{{"tool": "search_flights", "parameters": {{"origin": "NYC", "destination": "London", "date": "March 15"}}}}
{{"tool": "book_flight", "parameters": {{"flight_id": "AA123", "passenger_name": "John Smith", "seat_class": "economy"}}}}
{{"tool": "DONE", "parameters": {{}}}}

Return ONLY the JSON object."""
    
    response = completion(
        model="openai/gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1,
        api_key=OPENAI_API_KEY
    )
    
    decision_text = response.choices[0].message.content.strip()
    activity.logger.info(f"AI decision: {decision_text}")
    
    try:
        return json.loads(decision_text)
    except json.JSONDecodeError:
        if "DONE" in decision_text.upper():
            return {"tool": "DONE", "parameters": {}}
        return {"tool": decision_text, "parameters": {}}

print("AI decision activity with parameter extraction defined!")

@workflow.defn(sandboxed=False)  # sandboxed=False for Jupyter only
class AgenticWorkflow:
    """Workflow implementing an agentic loop with dynamic activities."""

    @workflow.run
    async def run(self, request: BookingRequest) -> BookingResult:
        """Main agentic loop - AI decides and executes autonomously."""
        
        from activities import agent_activities
        from tools import AVAILABLE_TOOLS
        
        context = ""
        steps_taken = []
        max_iterations = 10

        print(f"\nStarting agentic loop for: {request.goal}\n")

        # Main agentic loop
        for iteration in range(max_iterations):
            print(f"Iteration {iteration + 1}:")

            # AI decides which tool to use AND extracts parameters
            decision = await workflow.execute_activity(
                agent_activities.ai_select_tool_with_params,
                args=[request.goal, AVAILABLE_TOOLS, context],
                start_to_close_timeout=timedelta(seconds=30),
            )

            selected_tool = decision.get("tool", "")
            parameters = decision.get("parameters", {})

            # Check if done
            if selected_tool.upper() == "DONE":
                print("AI determined goal is complete!\n")
                break

            # Validate tool exists
            if selected_tool not in AVAILABLE_TOOLS:
                print(f"Unknown tool: {selected_tool}\n")
                continue

            # THIS IS THE MAGIC - Execute tool DYNAMICALLY by name!
            print(f"Executing: {selected_tool} with params: {parameters}")
            
            result = await workflow.execute_activity(
                selected_tool,  # Tool name as string becomes activity type!
                arg=parameters,  # Pass parameters to dynamic handler
                start_to_close_timeout=timedelta(seconds=30),
            )

            # Update context and track steps
            context += f"\n\nExecuted: {selected_tool}\nResult: {result}"
            steps_taken.append(f"{selected_tool}: {result[:100]}...")
            print(f"Result: {result[:100]}...\n")

        # Prepare final result
        if not steps_taken:
            steps_taken = ["No actions were taken"]

        success = any("book_flight" in step for step in steps_taken)
        message = f"{'Successfully completed' if success else 'Partially completed'}: {request.goal}"

        return BookingResult(
            message=message,
            steps_taken=steps_taken
        )

print("Agentic workflow with dynamic activities defined!")

In [ ]:
@activity.defn
async def search_flights(params: Dict[str, Union[str, int]]) -> str:
    """Search for available flights based on parameters."""
    origin = params.get("origin", "Unknown")
    destination = params.get("destination", "Unknown")
    date = params.get("date", "Unknown")
    
    activity.logger.info(f"Searching flights: {origin} → {destination} on {date}")
    
    return f"""Found 3 flights from {origin} to {destination} on {date}:
- Flight AA123: Departs 8:00 AM, arrives 8:00 PM, $450
- Flight UA456: Departs 2:15 PM, arrives 2:15 AM (+1 day), $380
- Flight DL789: Departs 6:30 PM, arrives 6:30 AM (+1 day), $420"""

@activity.defn
async def check_seat_availability(params: Dict[str, Union[str, int]]) -> str:
    """Check seat availability on a specific flight."""
    flight_id = params.get("flight_id", "Unknown")
    
    return f"""Seat availability for {flight_id}:
- Economy: 8 seats available
- Business: 4 seats available
- First: Sold out"""

@activity.defn
async def calculate_total_cost(params: Dict[str, Union[str, int]]) -> str:
    """Calculate total cost for a flight."""
    flight_id = params.get("flight_id", "UA456")
    passengers = params.get("passengers", 1)
    
    base_fare = 380
    taxes = 85
    total_per_person = base_fare + taxes
    grand_total = total_per_person * passengers
    
    return f"""Cost breakdown for {flight_id}:
- Base fare: ${base_fare} per person
- Taxes & fees: ${taxes} per person  
- Total per person: ${total_per_person}
- Grand total for {passengers} passenger(s): ${grand_total}"""

@activity.defn
async def book_flight(params: Dict[str, Union[str, int]]) -> str:
    """Book a specific flight."""
    flight_id = params.get("flight_id", "Unknown")
    passenger_name = params.get("passenger_name", "Guest")
    seat_class = params.get("seat_class", "economy")
    
    confirmation = f"CONF-{flight_id}-{datetime.now().strftime('%Y%m%d%H%M%S')}"
    return f"""Flight booked successfully!
- Flight: {flight_id}
- Passenger: {passenger_name}
- Class: {seat_class}
- Confirmation: {confirmation}"""

@activity.defn
async def send_confirmation(params: Dict[str, Union[str, int]]) -> str:
    """Send booking confirmation."""
    email = params.get("email", "passenger@example.com")
    confirmation_number = params.get("confirmation_number", "CONF-UNKNOWN")
    
    return f"Confirmation email sent to {email} with booking details: {confirmation_number}"

print("Tool activities defined with parameter support!")

## Step 5: The Magic - Dynamic Activity Handler

This is where the magic happens! We create a **single** dynamic activity that can handle **any** tool.

In [ ]:
# Create and start the worker - Notice we only register 2 activities!
import concurrent.futures

from temporalio.client import Client
from temporalio.worker import Worker
from activities import agent_activities, dynamic_tool_activity


async def run_worker():
    client = await Client.connect("localhost:7233")

    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        worker = Worker(
            client,
            task_queue="agentic-demo",
            workflows=[AgenticWorkflow],
            activities=[
                # Only need these TWO activities!
                agent_activities.ai_select_tool_with_params,  # AI decision maker
                dynamic_tool_activity,  # Dynamic handler for ALL tools!
            ],
            activity_executor=executor
        )

        print("Worker started with dynamic activities!")
        print("✅ No need to register each tool individually!")
        await worker.run()

# Start worker in background
worker_task = asyncio.create_task(run_worker())
await asyncio.sleep(2)  # Let worker start

## Step 6: The Agentic Workflow with Dynamic Execution

The workflow now uses dynamic activities - no hardcoded tool mappings!

In [ ]:
@workflow.defn(sandboxed=False)  # sandboxed=False for Jupyter only
class AgenticWorkflow:
    """Workflow implementing an agentic loop with dynamic activities."""
    
    @workflow.run
    async def run(self, request: BookingRequest) -> BookingResult:
        """Main agentic loop - AI decides and executes autonomously."""
        
        # Track execution context and steps
        context = ""
        steps_taken: List[str] = []
        max_iterations = 10
        
        workflow.logger.info(f"Starting agentic loop for goal: {request.goal}")
        
        # Main agentic loop
        for iteration in range(max_iterations):
            workflow.logger.info(f"Agentic loop iteration {iteration + 1}")
            
            # AI decides which tool to use AND extracts parameters
            decision = await workflow.execute_activity(
                ai_select_tool_with_params,
                args=[request.goal, AVAILABLE_TOOLS, context],
                start_to_close_timeout=timedelta(seconds=30),
            )
            
            tool_result = decision.get("tool", "")
            selected_tool = tool_result if isinstance(tool_result, str) else ""
            param_result = decision.get("parameters", {})
            parameters = param_result if isinstance(param_result, dict) else {}
            
            # Check if AI thinks we're done
            if selected_tool.upper() == "DONE":
                workflow.logger.info("AI determined goal is complete")
                break
            
            # Validate tool exists
            if selected_tool not in AVAILABLE_TOOLS:
                workflow.logger.error(f"AI selected unknown tool: {selected_tool}")
                context += f"\nError: Unknown tool '{selected_tool}'"
                continue
            
            # Execute the tool DYNAMICALLY using its name as the activity type!
            workflow.logger.info(f"Executing tool: {selected_tool} with params: {parameters}")
            
            try:
                # This is the magic - we pass the tool name as the activity type
                result: str = await workflow.execute_activity(
                    selected_tool,  # Tool name becomes the activity type!
                    arg=parameters,  # Parameters passed to the dynamic handler
                    start_to_close_timeout=timedelta(seconds=30),
                )
                
                # Update context with the result
                context += f"\n\nExecuted: {selected_tool}"
                if parameters:
                    params_str = ", ".join([f"{k}={v}" for k, v in parameters.items()])
                    context += f" with ({params_str})"
                context += f"\nResult: {result}"
                
                # Track the step
                step_desc = f"{selected_tool}"
                if parameters:
                    step_desc += f" ({', '.join([f'{k}={v}' for k, v in parameters.items()])})"
                step_desc += f": {result[:80]}..."
                steps_taken.append(step_desc)
                
                workflow.logger.info(f"Tool result: {result[:200]}...")
                
            except Exception as e:
                error_msg = f"Error executing {selected_tool}: {str(e)}"
                workflow.logger.error(error_msg)
                context += f"\n{error_msg}"
                steps_taken.append(f"{selected_tool} failed: {str(e)}")
        
        # Prepare final result
        if not steps_taken:
            steps_taken = ["No actions were taken"]
        
        # Determine success
        success = any("book_flight" in step and "failed" not in step for step in steps_taken)
        
        message = f"{'Successfully completed' if success else 'Partially completed'}: {request.goal}"
        
        return BookingResult(
            message=message,
            steps_taken=steps_taken
        )

print("Agentic workflow with dynamic activities defined!")

## How Dynamic Activities Work

When the workflow executes:
```python
await workflow.execute_activity(
    "search_flights",  # String tool name as activity type
    arg=parameters,    # Parameters for the tool
    ...
)
```

Here's what happens:

1. **Temporal sees unknown activity** - "search_flights" isn't registered
2. **Dynamic handler triggered** - `dynamic_tool_activity` is called
3. **Tool name extracted** - `activity.info().activity_type` gives us "search_flights"
4. **Handler lookup** - We find the function in `TOOL_HANDLERS`
5. **Execution** - The actual tool function runs with parameters
6. **Result returned** - Back to the workflow

This means:
- ✅ Add new tools without changing workflow/worker
- ✅ Tools can be loaded from external sources
- ✅ Single point of control for all tool execution
- ✅ Perfect for AI agents that discover tools at runtime!

## Step 7: Run the Demo

Let's see our dynamic agentic loop in action!

In [ ]:
# Start Temporal (if not already running)
import subprocess

try:
    temporal_process = subprocess.Popen(
        ["temporal", "server", "start-dev", "--ui-port", "8080"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    print("Temporal server starting...")
    await asyncio.sleep(3)
except:
    print("Temporal might already be running or not installed")

In [ ]:
# Create and start the worker with ONLY the dynamic activity!
import concurrent.futures
from temporalio.client import Client
from temporalio.worker import Worker

async def run_worker():
    client = await Client.connect("localhost:7233")
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        worker = Worker(
            client,
            task_queue="agentic-dynamic-demo",
            workflows=[AgenticWorkflow],
            activities=[
                ai_select_tool_with_params,
                dynamic_tool_activity,  # Only need to register the dynamic handler!
            ],
            activity_executor=executor
        )
        
        print("Worker started with dynamic activities!")
        await worker.run()

# Start worker in background
worker_task = asyncio.create_task(run_worker())
await asyncio.sleep(2)  # Let worker start

In [ ]:
# Now let's run our AI agent!
client = await Client.connect("localhost:7233")

# Define a goal
goal = "Book a flight from NYC to London for March 15th for John Smith"

print("=" * 60)
print(f"GOAL: {goal}")
print("=" * 60)
print("\nWatch as the AI autonomously:")
print("1. Searches for flights with extracted parameters")
print("2. Checks availability")
print("3. Calculates costs")
print("4. Books the flight")
print("5. Sends confirmation\n")

# Start the workflow
handle = await client.start_workflow(
    AgenticWorkflow.run,
    BookingRequest(goal=goal),
    id="agentic-dynamic-demo-1",
    task_queue="agentic-dynamic-demo"
)

# Wait for result
result = await handle.result()

print("=" * 60)
print("WORKFLOW COMPLETE!")
print("=" * 60)
print(f"\nResult: {result.message}")
print("\nSteps taken by AI agent:")
for i, step in enumerate(result.steps_taken, 1):
    print(f"   {i}. {step}")

## Understanding Dynamic Activities

### What Just Happened?

1. **Tool Selection**: AI chose `"search_flights"` as a string
2. **Dynamic Execution**: Workflow called `workflow.execute_activity("search_flights", ...)`
3. **Dynamic Handler**: The `dynamic_tool_activity` intercepted the unknown activity type
4. **Tool Lookup**: Handler found `"search_flights"` in the registry
5. **Execution**: Handler executed the actual tool function

### Benefits of Dynamic Activities

✅ **Extensibility**: Add new tools without modifying the workflow or worker

✅ **Flexibility**: Tools can be loaded from external sources or databases

✅ **Simplicity**: Only register one dynamic handler instead of every tool

✅ **Runtime Discovery**: Tools can be discovered and used at runtime

## Adding New Tools Dynamically

Let's add a new tool WITHOUT changing the workflow or restarting the worker!

In [ ]:
# Define a new tool
@activity.defn
async def check_weather(params: Dict[str, Union[str, int]]) -> str:
    """Check weather for the destination."""
    destination = params.get("destination", "Unknown")
    date = params.get("date", "Unknown")
    
    return f"""Weather forecast for {destination} on {date}:
- Temperature: 72°F (22°C)
- Conditions: Partly cloudy
- Precipitation: 10% chance"""

# Add to our tools registry
AVAILABLE_TOOLS["check_weather"] = ToolDefinition(
    name="check_weather",
    description="Check weather forecast for a destination",
    arguments=[
        ToolArgument("destination", "string", "Destination city"),
        ToolArgument("date", "string", "Date to check weather"),
    ]
)

# Add to handlers
TOOL_HANDLERS["check_weather"] = check_weather

print("✨ New tool added dynamically! The AI can now use 'check_weather'")

In [ ]:
# Test with a weather-related goal
weather_goal = "Check the weather in London and then book a flight from NYC for March 15"

handle = await client.start_workflow(
    AgenticWorkflow.run,
    BookingRequest(goal=weather_goal),
    id="weather-demo-1",
    task_queue="agentic-dynamic-demo"
)

result = await handle.result()

print(f"Goal: {weather_goal}")
print(f"Result: {result.message}")
print("\nThe AI used our new weather tool:")
for step in result.steps_taken:
    if "weather" in step.lower():
        print(f"  → {step}")

## Best Practices for Dynamic AI Agents

### 1. Tool Design
- **Clear Descriptions**: Help the AI understand what each tool does
- **Explicit Parameters**: Define parameter types and descriptions clearly
- **Idempotent Operations**: Tools should be safe to retry

### 2. Error Handling
- **Graceful Failures**: Return informative error messages
- **Validation**: Validate parameters before execution
- **Timeouts**: Set appropriate timeouts for each tool

### 3. Observability
- **Logging**: Log tool executions and decisions
- **Metrics**: Track tool usage and success rates
- **Tracing**: Use Temporal's built-in tracing

### 4. Security
- **Input Validation**: Always validate AI-generated parameters
- **Rate Limiting**: Limit tool execution frequency
- **Sandboxing**: Run untrusted tools in isolated environments

In [ ]:
import temporalio.common
from typing import Sequence

@activity.defn(dynamic=True)
async def dynamic_tool_activity(args: Sequence[temporalio.common.RawValue]) -> str:
    """Dynamic activity handler that executes tools based on activity type.
    
    This is called when the workflow executes an activity with an unknown type.
    It looks up the tool handler from the registry and executes it.
    """
    from tools import get_handler
    
    # Get the activity type which is our tool name
    tool_name = activity.info().activity_type
    
    # Extract the arguments from the raw value
    tool_args = activity.payload_converter().from_payload(args[0].payload, dict)
    
    activity.logger.info(f"Running dynamic tool '{tool_name}' with args: {tool_args}")
    
    # Delegate to the relevant function
    handler = get_handler(tool_name)
    if inspect.iscoroutinefunction(handler):
        result = await handler(tool_args)
    else:
        result = handler(tool_args)
    
    activity.logger.info(f"Tool '{tool_name}' result: {result}")
    return result

print("✨ Dynamic activity handler created - can execute ANY tool dynamically!")

## Key Takeaways

🎯 **Dynamic Activities** enable runtime tool execution without pre-registration

🤖 **AI + Parameters** - The AI extracts and passes parameters automatically

🔧 **Extensible** - Add new tools without changing workflow code

💪 **Durable** - Temporal ensures completion even with failures

👀 **Observable** - Every decision and execution is logged

⚡ **Scalable** - Run thousands of agents in parallel

### Why This Matters

This pattern represents the future of AI automation:
- **Autonomous**: AI makes its own decisions
- **Reliable**: Temporal provides durability
- **Flexible**: Easy to extend and modify
- **Production-Ready**: Built for real-world use

In [ ]:
# Clean up
worker_task.cancel()
try:
    temporal_process.terminate()
except:
    pass
print("Demo complete! 🎉")